In [1]:
#! pip install pyoso

In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [3]:
df_pkg = client.to_pandas("""
SELECT DISTINCT 
  package_artifact_source,
  package_artifact_name
FROM package_owners_v0 
WHERE package_owner_artifact_namespace = 'privacy-scaling-explorations'
""")
df_pkg

,package_artifact_source,package_artifact_name
0,NPM,anon-aadhaar-pcd
1,NPM,@zk-kit/incremental-merkle-tree
2,NPM,@zk-kit/imt
3,NPM,@zk-kit/circuits
4,NPM,@zk-kit/baby-jubjub
5,NPM,@zk-kit/eddsa-poseidon
6,NPM,@zk-kit/incremental-merkle-tree.sol
7,RUST,halo2curves
8,NPM,@excubiae/contracts
9,NPM,@zk-kit/logical-expressions


In [4]:
df_deps = client.to_pandas("""
SELECT DISTINCT
  CONCAT(s.from_artifact_namespace, '/', s.from_artifact_name) AS dependent,
  r.star_count AS stars,
  s.to_package_artifact_name AS package,
  s.to_package_artifact_source AS source
FROM sboms_v0 AS s
JOIN package_owners_v0 AS pkgs ON
  s.to_package_artifact_source = pkgs.package_artifact_source
  AND s.to_package_artifact_name = pkgs.package_artifact_name
JOIN repositories_v0 AS r
  ON s.from_artifact_id = r.artifact_id
WHERE package_owner_artifact_namespace = 'privacy-scaling-explorations'
ORDER BY 2 DESC  
""")
df_deps

,dependent,stars,package,source
0,kroma-network/tachyon,7767,halo2curves,RUST
1,zkonduit/ezkl,1074,halo2curves,RUST
2,zkonduit/ezkl,1074,halo2derive,RUST
3,semaphore-protocol/semaphore,975,@zk-kit/utils,NPM
4,semaphore-protocol/semaphore,975,@zk-kit/eddsa-poseidon,NPM
...,...,...,...,...
348,etherspot/hubbub,0,@zk-kit/utils,NPM
349,etherspot/hubbub,0,@zk-kit/eddsa-poseidon,NPM
350,etherspot/hubbub,0,@zk-kit/lean-imt,NPM
351,sociallayer-im/seastar-auth,0,@zk-kit/eddsa-poseidon,NPM


In [5]:
df_deps.to_csv('pse_deps.csv')